# Model Zoo

In this notebook you will see every kind of model in AutoProf. Printed in each cell will also be the list of parameters which the model looks for while fitting. Many models have unique capabilities and features, this will be introduced here, though fully taking advantage of them will be dependent on your science case.

For a family tree of all the AutoProf models see: link

Note, we will not be covering Compound_Model here as that requires a dedicated discussion. See the notebook: CombinedModels for that introduction.

In [ ]:
import autoprof as ap
import numpy as np
import torch
import matplotlib.pyplot as plt
%matplotlib inline
ap.AP_config.set_logging_output(stdout = True, filename = None) # see GettingStarted tutorial for what this does
basic_target = ap.image.Target_Image(np.zeros((100,100)), pixelscale = 1)

## Sky Models

### Flat Sky Model

In [ ]:
M = ap.models.AutoProf_Model(name = "flat sky", model_type = "flat sky model", parameters = {"center":[50,50], "sky": 1}, target = basic_target)
print(M.parameter_order())
print(tuple(M.parameters[P].units for P in M.parameter_order()))
M.initialize()

fig, ax = plt.subplots(figsize = (7,6))
ap.plots.model_image(fig, ax, M)
plt.show()

### Plane Sky Model

In [ ]:
M = ap.models.AutoProf_Model(name = "plane sky", model_type = "plane sky model", parameters = {"center":[50,50], "sky": 10, "delta": [1e-2,2e-2]}, target = basic_target)
print(M.parameter_order())
print(tuple(M.parameters[P].units for P in M.parameter_order()))
M.initialize()

fig, ax = plt.subplots(figsize = (7,6))
ap.plots.model_image(fig, ax, M)
plt.show()

## Star Models

### PSF Star

In [ ]:
from scipy.special import jv
xx, yy = np.meshgrid(np.linspace(-49,49,99), np.linspace(-49,49,99))
x = np.sqrt(xx**2 + yy**2)/5 +1e-6
PSF = (2*jv(1, x)/x)**2 + 1e-4 # the PSF can be any image, here we construct an airy disk
target = ap.image.Target_Image(data = np.zeros((100,100)), pixelscale = 1, psf = PSF) # the target image holds the PSF for itself

M = ap.models.AutoProf_Model(name = "psf star", model_type = "psf star model", target = target, parameters = {"center": [49.5,49.5], "flux": 1})
print(M.parameter_order())
print(tuple(M.parameters[P].units for P in M.parameter_order()))
M.initialize()

fig, ax = plt.subplots(1,2, figsize = (14,6))
ap.plots.model_image(fig, ax[0], M)
x = np.linspace(-49,49,99)/5 + 1e-6
ax[1].plot(x, np.log10((2*jv(1, x)/x)**2))
plt.show()

### Gaussian Star

In [ ]:
M = ap.models.AutoProf_Model(name = "gaussian star", model_type = "gaussian star model", parameters = {"center":[50,50], "sigma": 10, "flux": 1}, target = basic_target)
print(M.parameter_order())
print(tuple(M.parameters[P].units for P in M.parameter_order()))
M.initialize()

fig, ax = plt.subplots(1,2, figsize = (14,6))
ap.plots.model_image(fig, ax[0], M)
ax[1].plot(np.linspace(0,50,100), M.radial_model(torch.linspace(0,50,100)).detach().cpu().numpy())
plt.show()

### Moffat Star

In [ ]:
M = ap.models.AutoProf_Model(name = "moffat star", model_type = "moffat star model", parameters = {"center":[50,50], "n": 2., "Rd": 10., "I0": 1.}, target = basic_target)
print(M.parameter_order())
print(tuple(M.parameters[P].units for P in M.parameter_order()))
M.initialize()

fig, ax = plt.subplots(1,2, figsize = (14,6))
ap.plots.model_image(fig, ax[0], M)
ax[1].plot(np.linspace(0,50,100), M.radial_model(torch.linspace(0,50,100)).detach().cpu().numpy())
plt.show()

# Galaxy Models

### NonParametric Galaxy Model

In [ ]:
# Here we make an arbitrary nonparametric profile out of a sine wave and a line
x = np.linspace(0,10,14)
nonparametric_profile = np.sin(x*2+2)/20 + 1 - x/20
# Here we write down some corresponding radii for the points in the non-parametric profile. AutoProf will make 
# radii to match an input profile, but it is generally better to manually provide values so you have some control 
# over their placement. Just note that it is assumed the first point will be at R = 0.
NP_prof = [0] + list(np.logspace(np.log10(2),np.log10(50),13))

M = ap.models.AutoProf_Model(name = "nonparametric galaxy", model_type = "nonparametric galaxy model", parameters = {"center": [50,50], "q": 0.6, "PA": 60*np.pi/180, "I(R)": {"value": nonparametric_profile, "prof": NP_prof}}, target = basic_target)
print(M.parameter_order())
print(tuple(M.parameters[P].units for P in M.parameter_order()))
M.initialize()

fig, ax = plt.subplots(1,2, figsize = (14,6))
ap.plots.model_image(fig, ax[0], M)
ap.plots.galaxy_light_profile(fig,ax[1],M)
plt.show()

### Sersic Galaxy Model

In [ ]:
M = ap.models.AutoProf_Model(name = "sersic galaxy", model_type = "sersic galaxy model", parameters = {"center": [50,50], "q": 0.6, "PA": 60*np.pi/180, "n": 3, "Re": 10, "Ie": 1}, target = basic_target)
print(M.parameter_order())
print(tuple(M.parameters[P].units for P in M.parameter_order()))
M.initialize()

fig, ax = plt.subplots(1,2, figsize = (14,6))
ap.plots.model_image(fig, ax[0], M)
ap.plots.galaxy_light_profile(fig,ax[1],M)
plt.show()

### Exponential Galaxy Model

In [ ]:
M = ap.models.AutoProf_Model(name = "exponential galaxy", model_type = "exponential galaxy model", parameters = {"center": [50,50], "q": 0.6, "PA": 60*np.pi/180, "Re": 10, "Ie": 1}, target = basic_target)
print(M.parameter_order())
print(tuple(M.parameters[P].units for P in M.parameter_order()))
M.initialize()

fig, ax = plt.subplots(1,2, figsize = (14,6))
ap.plots.model_image(fig, ax[0], M)
ap.plots.galaxy_light_profile(fig,ax[1],M)
plt.show()

### Gaussian Galaxy Model

In [ ]:
M = ap.models.AutoProf_Model(name = "Gaussian", model_type = "gaussian galaxy model", parameters = {"center": [50,50], "q": 0.6, "PA": 60*np.pi/180, "sigma": 5, "flux": 1}, target = basic_target)
print(M.parameter_order())
print(tuple(M.parameters[P].units for P in M.parameter_order()))
M.initialize()

fig, ax = plt.subplots(1,2, figsize = (14,6))
ap.plots.model_image(fig, ax[0], M)
ap.plots.galaxy_light_profile(fig,ax[1],M)
plt.show()

### Nuker Galaxy Model

In [ ]:
M = ap.models.AutoProf_Model(name = "Nuker", model_type = "nuker galaxy model", parameters = {"center": [50,50], "q": 0.6, "PA": 60*np.pi/180, "Rb": 10., "Ib": 1., "alpha": 4., "beta": 3., "gamma": -0.2}, target = basic_target)
print(M.parameter_order())
print(tuple(M.parameters[P].units for P in M.parameter_order()))
M.initialize()

fig, ax = plt.subplots(1,2, figsize = (14,6))
ap.plots.model_image(fig, ax[0], M)
ap.plots.galaxy_light_profile(fig,ax[1],M)
plt.show()

## Super Ellipse Models

A super ellipse is a regular ellipse, except the radius metric changes from R = sqrt(x^2 + y^2) to the more general: R = (x^C + y^C)^1/C. The parameter C = 2 for a regular ellipse, for 0<C<2 the shape becomes more "disky" and for C > 2 the shape becomes more "boxy." In AutoProf we use the parameter C0 = C-2 for simplicity.

### NonParametric SuperEllipse

In [ ]:
M = ap.models.AutoProf_Model(name = "nonparametric superellipse", model_type = "nonparametric superellipse galaxy model", parameters = {"center": [50,50], "q": 0.6, "PA": 60*np.pi/180, "C0": 2, "I(R)": {"value": nonparametric_profile, "prof": NP_prof}}, target = basic_target)
print(M.parameter_order())
print(tuple(M.parameters[P].units for P in M.parameter_order()))
M.initialize()

fig, ax = plt.subplots(1,2, figsize = (14,6))
ap.plots.model_image(fig, ax[0], M)
ap.plots.galaxy_light_profile(fig,ax[1],M)
plt.show()

### Sersic SuperEllipse

In [ ]:
M = ap.models.AutoProf_Model(name = "sersic superellipse", model_type = "sersic superellipse galaxy model", parameters = {"center": [50,50], "q": 0.6, "PA": 60*np.pi/180, "C0": 2, "n": 3, "Re": 10, "Ie": 1}, target = basic_target)
print(M.parameter_order())
print(tuple(M.parameters[P].units for P in M.parameter_order()))
M.initialize()

fig, ax = plt.subplots(1,2, figsize = (14,6))
ap.plots.model_image(fig, ax[0], M)
ap.plots.galaxy_light_profile(fig,ax[1],M)
plt.show()

### Exponential SuperEllipse

In [ ]:
M = ap.models.AutoProf_Model(name = "exponential superellipse", model_type = "exponential superellipse galaxy model", parameters = {"center": [50,50], "q": 0.6, "PA": 60*np.pi/180, "C0": 2, "Re": 10, "Ie": 1}, target = basic_target)
print(M.parameter_order())
print(tuple(M.parameters[P].units for P in M.parameter_order()))
M.initialize()

fig, ax = plt.subplots(1,2, figsize = (14,6))
ap.plots.model_image(fig, ax[0], M)
ap.plots.galaxy_light_profile(fig,ax[1],M)
plt.show()

### Gaussian SuperEllipse

In [ ]:
M = ap.models.AutoProf_Model(name = "gaussian superellipse", model_type = "gaussian superellipse galaxy model", parameters = {"center": [50,50], "q": 0.6, "PA": 60*np.pi/180, "C0": 2, "sigma": 5, "flux": 1}, target = basic_target)
print(M.parameter_order())
print(tuple(M.parameters[P].units for P in M.parameter_order()))
M.initialize()

fig, ax = plt.subplots(1,2, figsize = (14,6))
ap.plots.model_image(fig, ax[0], M)
ap.plots.galaxy_light_profile(fig,ax[1],M)
plt.show()

### Nuker SuperEllipse

In [ ]:
M = ap.models.AutoProf_Model(name = "nuker superellipse", model_type = "nuker superellipse galaxy model", parameters = {"center": [50,50], "q": 0.6, "PA": 60*np.pi/180, "C0": 2, "Rb": 10., "Ib": 1., "alpha": 4., "beta": 3., "gamma": -0.2}, target = basic_target)
print(M.parameter_order())
print(tuple(M.parameters[P].units for P in M.parameter_order()))
M.initialize()

fig, ax = plt.subplots(1,2, figsize = (14,6))
ap.plots.model_image(fig, ax[0], M)
ap.plots.galaxy_light_profile(fig,ax[1],M)
plt.show()

## Fourier Ellipse Models

A Fourier ellipse is a scaling on the radius values as a function of theta. It takes the form: $R' = R * exp(\sum_m am*cos(m*theta + phim))$, where am and phim are the parameters which describe the Fourier perturbations. Using the "modes" argument as a tuple, users can select which Fourier modes are used. As a rough intuition: mode 1 acts like a shift of the model; mode 2 acts like ellipticity; mode 3 makes a lopsided model (triangular in the extreme); and mode 4 makes peanut/diamond perturbations. 

### NonParametric Fourier

In [ ]:
fourier_am = np.array([0.1, 0.3, -0.2])
fourier_phim = np.array([10*np.pi/180, 0, 40*np.pi/180])
M = ap.models.AutoProf_Model(name = "nonparametric fourier", model_type = "nonparametric fourier galaxy model", parameters = {"center": [50,50], "q": 0.6, "PA": 60*np.pi/180, "am": fourier_am, "phim": fourier_phim, "I(R)": {"value": nonparametric_profile, "prof": NP_prof}}, target = basic_target)
print(M.parameter_order())
print(tuple(M.parameters[P].units for P in M.parameter_order()))
M.initialize()

fig, ax = plt.subplots(1,2, figsize = (14,6))
ap.plots.model_image(fig, ax[0], M)
ap.plots.galaxy_light_profile(fig,ax[1],M)
plt.show()

### Sersic Fourier

In [ ]:
M = ap.models.AutoProf_Model(name = "sersic fourier", model_type = "sersic fourier galaxy model", parameters = {"center": [50,50], "q": 0.6, "PA": 60*np.pi/180, "am": fourier_am, "phim": fourier_phim, "n": 3, "Re": 10, "Ie": 1}, target = basic_target)
print(M.parameter_order())
print(tuple(M.parameters[P].units for P in M.parameter_order()))
M.initialize()

fig, ax = plt.subplots(1,2, figsize = (14,6))
ap.plots.model_image(fig, ax[0], M)
ap.plots.galaxy_light_profile(fig,ax[1],M)
plt.show()

### Exponential Fourier

In [ ]:
M = ap.models.AutoProf_Model(name = "exponential fourier", model_type = "exponential fourier galaxy model", parameters = {"center": [50,50], "q": 0.6, "PA": 60*np.pi/180, "am": fourier_am, "phim": fourier_phim, "Re": 10, "Ie": 1}, target = basic_target)
print(M.parameter_order())
print(tuple(M.parameters[P].units for P in M.parameter_order()))
M.initialize()

fig, ax = plt.subplots(1,2, figsize = (14,6))
ap.plots.model_image(fig, ax[0], M)
ap.plots.galaxy_light_profile(fig,ax[1],M)
plt.show()

### Gaussian Fourier

In [ ]:
M = ap.models.AutoProf_Model(name = "gaussian fourier", model_type = "gaussian fourier galaxy model", parameters = {"center": [50,50], "q": 0.6, "PA": 60*np.pi/180, "am": fourier_am, "phim": fourier_phim, "sigma": 5, "flux": 1}, target = basic_target)
print(M.parameter_order())
print(tuple(M.parameters[P].units for P in M.parameter_order()))
M.initialize()

fig, ax = plt.subplots(1,2, figsize = (14,6))
ap.plots.model_image(fig, ax[0], M)
ap.plots.galaxy_light_profile(fig,ax[1],M)
plt.show()

### Nuker Fourier

In [ ]:
M = ap.models.AutoProf_Model(name = "nuker fourier", model_type = "nuker fourier galaxy model", parameters = {"center": [50,50], "q": 0.6, "PA": 60*np.pi/180, "am": fourier_am, "phim": fourier_phim, "Rb": 10., "Ib": 1., "alpha": 4., "beta": 3., "gamma": -0.2}, target = basic_target)
print(M.parameter_order())
print(tuple(M.parameters[P].units for P in M.parameter_order()))
M.initialize()

fig, ax = plt.subplots(1,2, figsize = (14,6))
ap.plots.model_image(fig, ax[0], M)
ap.plots.galaxy_light_profile(fig,ax[1],M)
plt.show()

## Warp Model

A warp model performs a radially varying coordinate transform. Essentially instead of applying a rotation matrix **Rot** on all coordinates X,Y we instead construct a unique rotation matrix for each coordinate pair **Rot(R)** where $R = \sqrt(X^2 + Y^2)$. We also apply a radially dependent axis ratio **q(R)** to all the coordinates:

$R = \sqrt(X^2 + Y^2)$

$X, Y = Rotate(X, Y, PA(R))$

$Y = Y / q(R)$

The net effect is a radially varying PA and axis ratio which allows the model to represent spiral arms, bulges, or other features that change the apparent shape of a galaxy in a radially varying way.

### NonParametric Warp

In [ ]:
warp_q = np.linspace(0.1,0.4,14)
warp_pa = np.linspace(0,np.pi-0.2,14)
M = ap.models.AutoProf_Model(name = "nonparametric warp", model_type = "nonparametric warp galaxy model", parameters = {"center": [50,50], "q": 0.6, "PA": 60*np.pi/180, "q(R)": warp_q, "PA(R)": warp_pa, "I(R)": {"value": nonparametric_profile, "prof": NP_prof}}, target = basic_target)
print(M.parameter_order())
print(tuple(M.parameters[P].units for P in M.parameter_order()))
M.initialize()

fig, ax = plt.subplots(1,2, figsize = (14,6))
ap.plots.model_image(fig, ax[0], M)
ap.plots.galaxy_light_profile(fig,ax[1],M)
plt.show()

### Sersic Warp

In [ ]:
M = ap.models.AutoProf_Model(name = "sersic warp", model_type = "sersic warp galaxy model", parameters = {"center": [50,50], "q": 0.6, "PA": 60*np.pi/180, "q(R)": warp_q, "PA(R)": warp_pa, "n": 3, "Re": 10, "Ie": 1}, target = basic_target)
print(M.parameter_order())
print(tuple(M.parameters[P].units for P in M.parameter_order()))
M.initialize()

fig, ax = plt.subplots(1,2, figsize = (14,6))
ap.plots.model_image(fig, ax[0], M)
ap.plots.galaxy_light_profile(fig,ax[1],M)
plt.show()

### Exponential Warp

In [ ]:
M = ap.models.AutoProf_Model(name = "exp warp", model_type = "exponential warp galaxy model", parameters = {"center": [50,50], "q": 0.6, "PA": 60*np.pi/180, "q(R)": warp_q, "PA(R)": warp_pa, "Re": 10, "Ie": 1}, target = basic_target)
print(M.parameter_order())
print(tuple(M.parameters[P].units for P in M.parameter_order()))
M.initialize()

fig, ax = plt.subplots(1,2, figsize = (14,6))
ap.plots.model_image(fig, ax[0], M)
ap.plots.galaxy_light_profile(fig,ax[1],M)
plt.show()

### Gaussian Warp

In [ ]:
M = ap.models.AutoProf_Model(name = "gauss warp", model_type = "gaussian warp galaxy model", parameters = {"center": [50,50], "q": 0.6, "PA": 60*np.pi/180, "q(R)": warp_q, "PA(R)": warp_pa, "sigma": 5, "flux": 1}, target = basic_target)
print(M.parameter_order())
print(tuple(M.parameters[P].units for P in M.parameter_order()))
M.initialize()

fig, ax = plt.subplots(1,2, figsize = (14,6))
ap.plots.model_image(fig, ax[0], M)
ap.plots.galaxy_light_profile(fig,ax[1],M)
plt.show()

### Nuker Warp

In [ ]:
M = ap.models.AutoProf_Model(name = "nuker warp", model_type = "nuker warp galaxy model", parameters = {"center": [50,50], "q": 0.6, "PA": 60*np.pi/180, "q(R)": warp_q, "PA(R)": warp_pa, "Rb": 10., "Ib": 1., "alpha": 4., "beta": 3., "gamma": -0.2}, target = basic_target)
print(M.parameter_order())
print(tuple(M.parameters[P].units for P in M.parameter_order()))
M.initialize()

fig, ax = plt.subplots(1,2, figsize = (14,6))
ap.plots.model_image(fig, ax[0], M)
ap.plots.galaxy_light_profile(fig,ax[1],M)
plt.show()

## Ray Model

A ray model allows the user to break the galaxy up into regions that can be fit separately. There are two basic kinds of ray model: symmetric and asymetric. A symmetric ray model (symmetric_rays = True) assumes 180 degree symmetry of the galaxy and so each ray is reflected through the center. This means that essentially the major axes and the minor axes are being fit separately. For an asymmetric ray model (symmetric_rays = False) each ray is it's own profile to be fit separately. 

In a ray model there is a smooth boundary between the rays. This smoothness is acomplished by applying a $(\cos(r*theta)+1)/2$ weight to each profile, where r is dependent on the number of rays and theta is shifted to center on each ray in turn. The exact cosine weighting is dependent on if the rays are symmetric and if there is an even or odd number of rays. 

### NonParametric Ray

In [ ]:
M = ap.models.AutoProf_Model(name = "nonparametric ray", model_type = "nonparametric ray galaxy model", symmetric_rays = True, rays = 2, parameters = {"center": [50,50], "q": 0.6, "PA": 60*np.pi/180, "I(R)": {"value": np.array([nonparametric_profile, nonparametric_profile*2]), "prof": NP_prof}}, target = basic_target)
print(M.parameter_order())
print(tuple(M.parameters[P].units for P in M.parameter_order()))
M.initialize()

fig, ax = plt.subplots(1,2, figsize = (14,6))
ap.plots.model_image(fig, ax[0], M)
ap.plots.ray_light_profile(fig,ax[1],M)
plt.show()

### Sersic Ray

In [ ]:
M = ap.models.AutoProf_Model(name = "sersic ray", model_type = "sersic ray galaxy model", symmetric_rays = True, rays = 2, parameters = {"center": [50,50], "q": 0.6, "PA": 60*np.pi/180, "n": [1,3], "Re": [10,5],"Ie": [1,0.5]}, target = basic_target)
print(M.parameter_order())
print(tuple(M.parameters[P].units for P in M.parameter_order()))
M.initialize()

fig, ax = plt.subplots(1,2, figsize = (14,6))
ap.plots.model_image(fig, ax[0], M)
ap.plots.ray_light_profile(fig,ax[1],M)
plt.show()

### Exponential Ray

In [ ]:
M = ap.models.AutoProf_Model(name = "exponential ray", model_type = "exponential ray galaxy model", symmetric_rays = True, rays = 2, parameters = {"center": [50,50], "q": 0.6, "PA": 60*np.pi/180, "Re": [10,5], "Ie": [1,2]}, target = basic_target)
print(M.parameter_order())
print(tuple(M.parameters[P].units for P in M.parameter_order()))
M.initialize()

fig, ax = plt.subplots(1,2, figsize = (14,6))
ap.plots.model_image(fig, ax[0], M)
ap.plots.ray_light_profile(fig,ax[1],M)
plt.show()

### Gaussian Ray

In [ ]:
M = ap.models.AutoProf_Model(name = "gaussian ray", model_type = "gaussian ray galaxy model", symmetric_rays = True, rays = 2, parameters = {"center": [50,50], "q": 0.6, "PA": 60*np.pi/180, "sigma": [5,8], "flux": [1.5,1.]}, target = basic_target)
print(M.parameter_order())
print(tuple(M.parameters[P].units for P in M.parameter_order()))
M.initialize()

fig, ax = plt.subplots(1,2, figsize = (14,6))
ap.plots.model_image(fig, ax[0], M)
ap.plots.ray_light_profile(fig,ax[1],M)
plt.show()

## Wedge Model

A wedge model behaves just like a ray model, except the boundaries are sharp. This has the advantage that the wedges can be very different in brightness without the "smoothing" from the ray model washing out the dimmer one. It also has the advantage of less "mixing" of information between the rays, each one can be counted on to have fit only the pixels in it's wedge without any influence from a neighbor. However, it has the disadvantage that the discontinuity at the boundary makes fitting behave strangely when a bright spot lays near the boundary.

### NonParametric Wedge

In [ ]:
M = ap.models.AutoProf_Model(name = "nonparametric wedge", model_type = "nonparametric wedge galaxy model", symmetric_wedges = True, wedges = 2, parameters = {"center": [50,50], "q": 0.6, "PA": 60*np.pi/180, "I(R)": {"value": np.array([nonparametric_profile, nonparametric_profile*2]), "prof": NP_prof}}, target = basic_target)
print(M.parameter_order())
print(tuple(M.parameters[P].units for P in M.parameter_order()))
M.initialize()

fig, ax = plt.subplots(1,2, figsize = (14,6))
ap.plots.model_image(fig, ax[0], M)
ap.plots.wedge_light_profile(fig,ax[1],M)
plt.show()

# High Order Warp Models

The models below combine the Warp coordinate transform with radial behaviour transforms: SuperEllipse and Fourier. These higher order models can create higly complex shapes, though their scientific use-case is less clear. They are included for completeness as they may be useful in some specific instances. These models are also included to demonstrate the flexibility in making AutoProf models, in a future tutorial we will discuss how to make your own model types.

### NonParametric SuperEllipse Warp

In [ ]:
M = ap.models.AutoProf_Model(name = "nonparametric super warp", model_type = "nonparametric superellipse warp galaxy model", parameters = {"center": [50,50], "q": 0.6, "PA": 60*np.pi/180, "q(R)": warp_q, "PA(R)": warp_pa, "C0": 2, "I(R)": {"value": nonparametric_profile, "prof": NP_prof}}, target = basic_target)
print(M.parameter_order())
print(tuple(M.parameters[P].units for P in M.parameter_order()))
M.initialize()

fig, ax = plt.subplots(1,2, figsize = (14,6))
ap.plots.model_image(fig, ax[0], M)
ap.plots.galaxy_light_profile(fig,ax[1],M)
plt.show()

### Sersic SuperEllipse Warp

In [ ]:
M = ap.models.AutoProf_Model(name = "sersic super warp", model_type = "sersic superellipse warp galaxy model", parameters = {"center": [50,50], "q": 0.6, "PA": 60*np.pi/180, "q(R)": warp_q, "PA(R)": warp_pa, "C0": 2, "n": 3, "Re": 10, "Ie": 1}, target = basic_target)
print(M.parameter_order())
print(tuple(M.parameters[P].units for P in M.parameter_order()))
M.initialize()

fig, ax = plt.subplots(1,2, figsize = (14,6))
ap.plots.model_image(fig, ax[0], M)
ap.plots.galaxy_light_profile(fig,ax[1],M)
plt.show()

### Exponential SuperEllipse Warp

In [ ]:
M = ap.models.AutoProf_Model(name = "exponential super warp", model_type = "exponential superellipse warp galaxy model", parameters = {"center": [50,50], "q": 0.6, "PA": 60*np.pi/180, "q(R)": warp_q, "PA(R)": warp_pa, "C0": 2, "Re": 10, "Ie": 1}, target = basic_target)
print(M.parameter_order())
print(tuple(M.parameters[P].units for P in M.parameter_order()))
M.initialize()

fig, ax = plt.subplots(1,2, figsize = (14,6))
ap.plots.model_image(fig, ax[0], M)
ap.plots.galaxy_light_profile(fig,ax[1],M)
plt.show()

### Gaussian SuperEllipse Warp

In [ ]:
M = ap.models.AutoProf_Model(name = "gauss super warp", model_type = "gaussian superellipse warp galaxy model", parameters = {"center": [50,50], "q": 0.6, "PA": 60*np.pi/180, "q(R)": warp_q, "PA(R)": warp_pa, "C0": 2, "sigma": 5, "flux": 1}, target = basic_target)
print(M.parameter_order())
print(tuple(M.parameters[P].units for P in M.parameter_order()))
M.initialize()

fig, ax = plt.subplots(1,2, figsize = (14,6))
ap.plots.model_image(fig, ax[0], M)
ap.plots.galaxy_light_profile(fig,ax[1],M)
plt.show()

### NonParametric Fourier Warp

not sure how this abomination would fit a galaxy, but you are welcome to try

In [ ]:
M = ap.models.AutoProf_Model(name = "nonparametric fourier warp", model_type = "nonparametric fourier warp galaxy model", modes = (1,3,4), parameters = {"center": [50,50], "q": 0.6, "PA": 60*np.pi/180, "q(R)": warp_q, "PA(R)": warp_pa, "am": fourier_am, "phim": fourier_phim, "I(R)": {"value": nonparametric_profile, "prof": NP_prof}}, target = basic_target)
print(M.parameter_order())
print(tuple(M.parameters[P].units for P in M.parameter_order()))
M.initialize()

fig, ax = plt.subplots(1,2, figsize = (14,6))
ap.plots.model_image(fig, ax[0], M)
ap.plots.galaxy_light_profile(fig,ax[1],M)
plt.show()

### Sersic Fourier Warp

In [ ]:
M = ap.models.AutoProf_Model(name = "sersic fourier warp", model_type = "sersic fourier warp galaxy model", parameters = {"center": [50,50], "q": 0.6, "PA": 60*np.pi/180, "q(R)": warp_q, "PA(R)": warp_pa, "am": fourier_am, "phim": fourier_phim, "n": 3, "Re": 10, "Ie": 1}, target = basic_target)
print(M.parameter_order())
print(tuple(M.parameters[P].units for P in M.parameter_order()))
M.initialize()

fig, ax = plt.subplots(1,2, figsize = (14,6))
ap.plots.model_image(fig, ax[0], M)
ap.plots.galaxy_light_profile(fig,ax[1],M)
plt.show()

### Exponential Fourier Warp

In [ ]:
M = ap.models.AutoProf_Model(name = "exp fourier warp", model_type = "exponential fourier warp galaxy model", parameters = {"center": [50,50], "q": 0.6, "PA": 60*np.pi/180, "q(R)": warp_q, "PA(R)": warp_pa, "am": fourier_am, "phim": fourier_phim, "Re": 10, "Ie": 1}, target = basic_target)
print(M.parameter_order())
print(tuple(M.parameters[P].units for P in M.parameter_order()))
M.initialize()

fig, ax = plt.subplots(1,2, figsize = (14,6))
ap.plots.model_image(fig, ax[0], M)
ap.plots.galaxy_light_profile(fig,ax[1],M)
plt.show()

### Gassian Fourier Warp

In [ ]:
M = ap.models.AutoProf_Model(name = "gaussian fourier warp", model_type = "gaussian fourier warp galaxy model", parameters = {"center": [50,50], "q": 0.6, "PA": 60*np.pi/180, "q(R)": warp_q, "PA(R)": warp_pa, "am": fourier_am, "phim": fourier_phim, "sigma": 5, "flux": 1}, target = basic_target)
print(M.parameter_order())
print(tuple(M.parameters[P].units for P in M.parameter_order()))
M.initialize()

fig, ax = plt.subplots(1,2, figsize = (14,6))
ap.plots.model_image(fig, ax[0], M)
ap.plots.galaxy_light_profile(fig,ax[1],M)
plt.show()